In [ ]:
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-11.2/nvvm/libdevice
/usr/local/cuda-11.2/nvvm-prev/libdevice
find: ‘/proc/60/task/60/net’: Invalid argument
find: ‘/proc/60/net’: Invalid argument
/usr/local/cuda-11.2/nvvm/lib64/libnvvm.so
/usr/local/cuda-11.2/nvvm-prev/lib64/libnvvm.so
find: ‘/proc/60/task/60/net’: Invalid argument
find: ‘/proc/60/net’: Invalid argument
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 43.5 MB/s eta 0:00:00


In [ ]:
#Loading part
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
downloaded = drive.CreateFile({'id': '1pQxX_uzZFz2p07c5BCbrlfRKXVcCxV60'})   # replace the id with id of file you want to access
downloaded.GetContentFile('ExtractedRGBOulu_NPU.zip')        # replace the file name with your file
!unzip ExtractedRGBOulu_NPU

MessageError: ignored

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from scipy import signal
import scipy.io
import math
import sys
import time
from IPython.display import HTML, display
from IPython.display import clear_output
import PIL
from PIL import Image
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8
import mahotas
import mahotas.demos
from pylab import gray, imshow, show
import numpy as np

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)            
    return allFiles

In [ ]:
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as tf
import skimage
import skimage.color as skcolor
import skimage.io as skio
from PIL import Image
import cv2
import numpy as np
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
import matplotlib.pylab as plt

In [ ]:
def show(im):
    plt.imshow(im)
    plt.show()
    

In [ ]:
def rgb2grayscale(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    grayscale = im.dot(xform.T)
    grayscale[:,:,[1,2]] += 128
    return np.uint8(grayscale)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from numba import int32, float32    # import the types
from numba.experimental import jitclass


class Human:

    
    def __init__(self):
        self.img = 0
    
    
    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    
    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        height, width, xx = image.shape
        img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
       
        img_lbp = np.zeros((height, width), np.uint8)

        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbp[i, j] = Human.lbp_calculated_pixel8(img_gray, i, j)

        path = '/content/GRY_LBP/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/GRY_LBP/dev'
        os.chdir(directory)
        cv2.imwrite(strr[34:], img_lbp)
#         cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human = Human()

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/dev")

# len(path_list)
for i in path_list:
  human.rgblbp(i)

In [ ]:
!zip -r /content/dev.zip /content/dev
from google.colab import files
files.download("/content/dev.zip")